In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from skimage.feature import canny
from sklearn.preprocessing import LabelEncoder

In [2]:
# EDGE BASED SEGMENTATION
def preprocess1(image):
  # resizing
  image = cv2.resize(image,(96,96))
  # gray scaling
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#   # log transformation
#   c=0
#   if np.max(image) == 0:
#         c=0
#   else:
#         c = 255 / np.log(1 + np.max(image)) 
#   log_image = c * (np.log(image + 1)) 
#   log_image = np.array(log_image, dtype = np.uint8)  
#   # canny edge detection
#   image = canny(log_image)
  return image

In [3]:
rootdir = 'asl_alphabet/asl_alphabet_train/asl_alphabet_train'
images = []
labels = []

for folder in os.listdir(rootdir):
    classes = os.path.join(rootdir,folder)
    label = folder
    for file in os.listdir(classes):
        img = os.path.join(classes, file)
        image = cv2.imread(img)
        image = preprocess1(image)
        images.append(image)
        labels.append(label)

encoder = LabelEncoder()
labels = encoder.fit_transform(labels)


In [4]:
X = np.array(images)
y = np.array(labels) 

In [5]:
from sklearn.model_selection import train_test_split
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# Step 1: Feature Extraction
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    # Check if descriptors is not None and has at least one row
    if descriptors is not None and descriptors.shape[0] > 0:
        return descriptors
    else:
        return np.zeros((1, 128), dtype=np.float32) # Return empty array of shape (0, 128)

# Step 2: Feature Clustering
def learn_visual_words(train_images, num_words):
    # Concatenate features from all images
    descriptors = np.concatenate([extract_features(img) for img in train_images])
    # Cluster features into visual words
    kmeans = KMeans(n_clusters=num_words, random_state=0).fit(descriptors.astype(np.float32))
    return kmeans

# Step 3: Feature Quantization
def quantize_features(features, kmeans):
    visual_words = kmeans.predict(features)
    return visual_words.astype(np.int32)

# Step 4: Histogram Creation
def create_histogram(visual_words, num_words):
    histogram = np.zeros(num_words, dtype=np.float32)
    visual_words = visual_words.astype(np.int32)  # convert visual_words to int32
    for word in visual_words:
        histogram[word] += 1
    return histogram

# Step 5: Classifier Training and Testing
def train_classifier(train_histograms, train_labels):
    svm = LinearSVC()
    svm.fit(train_histograms.astype(np.float32), train_labels.astype(np.float32))
    return svm

def test_classifier(svm, test_histograms, test_labels):
    pred_labels = svm.predict(test_histograms.astype(np.float32))
    print(classification_report(test_labels.astype(np.float32), pred_labels.astype(np.float32)))
    accuracy = accuracy_score(test_labels.astype(np.float32), pred_labels.astype(np.float32))
    return accuracy

# Load train and test images
train_images = X_train
train_labels = y_train

test_images = X_test
test_labels = y_test

# Learn visual words
num_words = 200
kmeans = learn_visual_words(train_images, num_words)


In [7]:
# Compute histograms for train and test images
train_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in train_images])
test_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in test_images])

# Train and test SVM classifier
svm = train_classifier(train_histograms, train_labels)
accuracy = test_classifier(svm, test_histograms, test_labels)

print("Accuracy: %.2f%%" % (accuracy * 100))

              precision    recall  f1-score   support

         0.0       0.79      0.81      0.80       883
         1.0       0.76      0.80      0.78       966
         2.0       0.84      0.85      0.85       972
         3.0       0.86      0.83      0.85       892
         4.0       0.75      0.71      0.73       901
         5.0       0.90      0.90      0.90       847
         6.0       0.81      0.83      0.82       907
         7.0       0.81      0.88      0.84       851
         8.0       0.78      0.77      0.77       898
         9.0       0.76      0.82      0.79       881
        10.0       0.85      0.82      0.84       934
        11.0       0.85      0.89      0.87       904
        12.0       0.64      0.64      0.64       853
        13.0       0.68      0.60      0.64       900
        14.0       0.71      0.72      0.72       878
        15.0       0.71      0.77      0.74       866
        16.0       0.75      0.79      0.77       873
        17.0       0.69    

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [6]:
# with 100 words
# accuracy with log transformation - 55% 
# accuracy w/o = 59.08%

# with 200 words
# accuracy w/o = 75.25%

# with 300 words
# accuracy w/o = 82.35%

# with 500 words
# accuracy w/o = 89.23%

# with 700 words
# accuracy w/o = 92.69%

# with 1000 words
# accuracy w/o = 94.77%

In [6]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# Step 1: Feature Extraction
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    # Check if descriptors is not None and has at least one row
    if descriptors is not None and descriptors.shape[0] > 0:
        return descriptors
    else:
        return np.zeros((1, 128), dtype=np.float32) # Return empty array of shape (0, 128)

# Step 2: Feature Clustering
def learn_visual_words(train_images, num_words):
    # Concatenate features from all images
    descriptors = np.concatenate([extract_features(img) for img in train_images])
    # Cluster features into visual words
    kmeans = KMeans(n_clusters=num_words, random_state=0).fit(descriptors.astype(np.float32))
    return kmeans

# Step 3: Feature Quantization
def quantize_features(features, kmeans):
    visual_words = kmeans.predict(features)
    return visual_words.astype(np.int32)

# Step 4: Histogram Creation
def create_histogram(visual_words, num_words):
    histogram = np.zeros(num_words, dtype=np.float32)
    visual_words = visual_words.astype(np.int32)  # convert visual_words to int32
    for word in visual_words:
        histogram[word] += 1
    return histogram

# Step 5: Classifier Training and Testing
def train_classifier(train_histograms, train_labels):
    svm = LinearSVC()
    svm.fit(train_histograms.astype(np.float32), train_labels.astype(np.float32))
    return svm

def test_classifier(svm, test_histograms, test_labels):
    pred_labels = svm.predict(test_histograms.astype(np.float32))
    print(classification_report(test_labels.astype(np.float32), pred_labels.astype(np.float32)))
    accuracy = accuracy_score(test_labels.astype(np.float32), pred_labels.astype(np.float32))
    return accuracy

# Load train and test images
train_images = X_train
train_labels = y_train

test_images = X_test
test_labels = y_test

# Learn visual words
num_words = 300
kmeans = learn_visual_words(train_images, num_words)


In [7]:
# Compute histograms for train and test images
train_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in train_images])
test_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in test_images])

# Train and test SVM classifier
svm = train_classifier(train_histograms, train_labels)
accuracy = test_classifier(svm, test_histograms, test_labels)

print("Accuracy: %.2f%%" % (accuracy * 100))

              precision    recall  f1-score   support

         0.0       0.85      0.85      0.85       883
         1.0       0.85      0.87      0.86       966
         2.0       0.89      0.90      0.89       972
         3.0       0.91      0.89      0.90       892
         4.0       0.82      0.79      0.80       901
         5.0       0.94      0.96      0.95       847
         6.0       0.89      0.88      0.88       907
         7.0       0.89      0.92      0.91       851
         8.0       0.85      0.83      0.84       898
         9.0       0.85      0.88      0.87       881
        10.0       0.90      0.89      0.89       934
        11.0       0.91      0.93      0.92       904
        12.0       0.70      0.73      0.72       853
        13.0       0.77      0.72      0.74       900
        14.0       0.79      0.83      0.81       878
        15.0       0.80      0.80      0.80       866
        16.0       0.82      0.83      0.83       873
        17.0       0.76    

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [6]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# Step 1: Feature Extraction
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    # Check if descriptors is not None and has at least one row
    if descriptors is not None and descriptors.shape[0] > 0:
        return descriptors
    else:
        return np.zeros((1, 128), dtype=np.float32) # Return empty array of shape (0, 128)

# Step 2: Feature Clustering
def learn_visual_words(train_images, num_words):
    # Concatenate features from all images
    descriptors = np.concatenate([extract_features(img) for img in train_images])
    # Cluster features into visual words
    kmeans = KMeans(n_clusters=num_words, random_state=0).fit(descriptors.astype(np.float32))
    return kmeans

# Step 3: Feature Quantization
def quantize_features(features, kmeans):
    visual_words = kmeans.predict(features)
    return visual_words.astype(np.int32)

# Step 4: Histogram Creation
def create_histogram(visual_words, num_words):
    histogram = np.zeros(num_words, dtype=np.float32)
    visual_words = visual_words.astype(np.int32)  # convert visual_words to int32
    for word in visual_words:
        histogram[word] += 1
    return histogram

# Step 5: Classifier Training and Testing
def train_classifier(train_histograms, train_labels):
    svm = LinearSVC()
    svm.fit(train_histograms.astype(np.float32), train_labels.astype(np.float32))
    return svm

def test_classifier(svm, test_histograms, test_labels):
    pred_labels = svm.predict(test_histograms.astype(np.float32))
    print(classification_report(test_labels.astype(np.float32), pred_labels.astype(np.float32)))
    accuracy = accuracy_score(test_labels.astype(np.float32), pred_labels.astype(np.float32))
    return accuracy

# Load train and test images
train_images = X_train
train_labels = y_train

test_images = X_test
test_labels = y_test

# Learn visual words
num_words = 500
kmeans = learn_visual_words(train_images, num_words)


In [7]:
# Compute histograms for train and test images
train_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in train_images])
test_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in test_images])

# Train and test SVM classifier
svm = train_classifier(train_histograms, train_labels)
accuracy = test_classifier(svm, test_histograms, test_labels)

print("Accuracy: %.2f%%" % (accuracy * 100))

              precision    recall  f1-score   support

         0.0       0.90      0.92      0.91       883
         1.0       0.91      0.91      0.91       966
         2.0       0.93      0.94      0.94       972
         3.0       0.94      0.94      0.94       892
         4.0       0.91      0.89      0.90       901
         5.0       0.96      0.98      0.97       847
         6.0       0.93      0.93      0.93       907
         7.0       0.94      0.96      0.95       851
         8.0       0.91      0.92      0.91       898
         9.0       0.90      0.93      0.92       881
        10.0       0.94      0.91      0.93       934
        11.0       0.95      0.96      0.95       904
        12.0       0.81      0.83      0.82       853
        13.0       0.83      0.82      0.82       900
        14.0       0.89      0.89      0.89       878
        15.0       0.90      0.90      0.90       866
        16.0       0.89      0.88      0.88       873
        17.0       0.86    

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [6]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# Step 1: Feature Extraction
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    # Check if descriptors is not None and has at least one row
    if descriptors is not None and descriptors.shape[0] > 0:
        return descriptors
    else:
        return np.zeros((1, 128), dtype=np.float32) # Return empty array of shape (0, 128)

# Step 2: Feature Clustering
def learn_visual_words(train_images, num_words):
    # Concatenate features from all images
    descriptors = np.concatenate([extract_features(img) for img in train_images])
    # Cluster features into visual words
    kmeans = KMeans(n_clusters=num_words, random_state=0).fit(descriptors.astype(np.float32))
    return kmeans

# Step 3: Feature Quantization
def quantize_features(features, kmeans):
    visual_words = kmeans.predict(features)
    return visual_words.astype(np.int32)

# Step 4: Histogram Creation
def create_histogram(visual_words, num_words):
    histogram = np.zeros(num_words, dtype=np.float32)
    visual_words = visual_words.astype(np.int32)  # convert visual_words to int32
    for word in visual_words:
        histogram[word] += 1
    return histogram

# Step 5: Classifier Training and Testing
def train_classifier(train_histograms, train_labels):
    svm = LinearSVC()
    svm.fit(train_histograms.astype(np.float32), train_labels.astype(np.float32))
    return svm

def test_classifier(svm, test_histograms, test_labels):
    pred_labels = svm.predict(test_histograms.astype(np.float32))
    print(classification_report(test_labels.astype(np.float32), pred_labels.astype(np.float32)))
    accuracy = accuracy_score(test_labels.astype(np.float32), pred_labels.astype(np.float32))
    return accuracy

# Load train and test images
train_images = X_train
train_labels = y_train

test_images = X_test
test_labels = y_test

# Learn visual words
num_words = 700
kmeans = learn_visual_words(train_images, num_words)


In [7]:
# Compute histograms for train and test images
train_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in train_images])
test_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in test_images])

# Train and test SVM classifier
svm = train_classifier(train_histograms, train_labels)
accuracy = test_classifier(svm, test_histograms, test_labels)

print("Accuracy: %.2f%%" % (accuracy * 100))

              precision    recall  f1-score   support

         0.0       0.94      0.95      0.94       883
         1.0       0.93      0.92      0.93       966
         2.0       0.97      0.96      0.97       972
         3.0       0.96      0.95      0.96       892
         4.0       0.92      0.93      0.92       901
         5.0       0.98      0.99      0.98       847
         6.0       0.96      0.97      0.97       907
         7.0       0.97      0.97      0.97       851
         8.0       0.95      0.92      0.93       898
         9.0       0.96      0.95      0.95       881
        10.0       0.96      0.95      0.95       934
        11.0       0.96      0.97      0.97       904
        12.0       0.89      0.87      0.88       853
        13.0       0.87      0.87      0.87       900
        14.0       0.92      0.94      0.93       878
        15.0       0.93      0.94      0.94       866
        16.0       0.94      0.94      0.94       873
        17.0       0.89    

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [9]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# Step 1: Feature Extraction
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    # Check if descriptors is not None and has at least one row
    if descriptors is not None and descriptors.shape[0] > 0:
        return descriptors
    else:
        return np.zeros((1, 128), dtype=np.float32) # Return empty array of shape (0, 128)

# Step 2: Feature Clustering
def learn_visual_words(train_images, num_words):
    # Concatenate features from all images
    descriptors = np.concatenate([extract_features(img) for img in train_images])
    # Cluster features into visual words
    kmeans = KMeans(n_clusters=num_words, random_state=0).fit(descriptors.astype(np.float32))
    return kmeans

# Step 3: Feature Quantization
def quantize_features(features, kmeans):
    visual_words = kmeans.predict(features)
    return visual_words.astype(np.int32)

# Step 4: Histogram Creation
def create_histogram(visual_words, num_words):
    histogram = np.zeros(num_words, dtype=np.float32)
    visual_words = visual_words.astype(np.int32)  # convert visual_words to int32
    for word in visual_words:
        histogram[word] += 1
    return histogram

# Step 5: Classifier Training and Testing
def train_classifier(train_histograms, train_labels):
    svm = LinearSVC()
    svm.fit(train_histograms.astype(np.float32), train_labels.astype(np.float32))
    return svm

def test_classifier(svm, test_histograms, test_labels):
    pred_labels = svm.predict(test_histograms.astype(np.float32))
    print(classification_report(test_labels.astype(np.float32), pred_labels.astype(np.float32)))
    accuracy = accuracy_score(test_labels.astype(np.float32), pred_labels.astype(np.float32))
    return accuracy

# Load train and test images
train_images = X_train
train_labels = y_train

test_images = X_test
test_labels = y_test

# Learn visual words
num_words = 1000
kmeans = learn_visual_words(train_images, num_words)


In [10]:
# Compute histograms for train and test images
train_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in train_images])
test_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in test_images])

# Train and test SVM classifier
svm = train_classifier(train_histograms, train_labels)
accuracy = test_classifier(svm, test_histograms, test_labels)

print("Accuracy: %.2f%%" % (accuracy * 100))

              precision    recall  f1-score   support

         0.0       0.95      0.95      0.95       883
         1.0       0.94      0.94      0.94       966
         2.0       0.97      0.97      0.97       972
         3.0       0.98      0.96      0.97       892
         4.0       0.95      0.95      0.95       901
         5.0       0.97      0.99      0.98       847
         6.0       0.96      0.98      0.97       907
         7.0       0.97      0.98      0.97       851
         8.0       0.96      0.95      0.96       898
         9.0       0.97      0.97      0.97       881
        10.0       0.96      0.96      0.96       934
        11.0       0.97      0.99      0.98       904
        12.0       0.91      0.90      0.91       853
        13.0       0.92      0.91      0.91       900
        14.0       0.93      0.94      0.94       878
        15.0       0.95      0.95      0.95       866
        16.0       0.95      0.96      0.96       873
        17.0       0.93    

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [6]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# Step 1: Feature Extraction
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    # Check if descriptors is not None and has at least one row
    if descriptors is not None and descriptors.shape[0] > 0:
        return descriptors
    else:
        return np.zeros((1, 128), dtype=np.float32) # Return empty array of shape (0, 128)

# Step 2: Feature Clustering
def learn_visual_words(train_images, num_words):
    # Concatenate features from all images
    descriptors = np.concatenate([extract_features(img) for img in train_images])
    # Cluster features into visual words
    kmeans = KMeans(n_clusters=num_words, random_state=0).fit(descriptors.astype(np.float32))
    return kmeans

# Step 3: Feature Quantization
def quantize_features(features, kmeans):
    visual_words = kmeans.predict(features)
    return visual_words.astype(np.int32)

# Step 4: Histogram Creation
def create_histogram(visual_words, num_words):
    histogram = np.zeros(num_words, dtype=np.float32)
    visual_words = visual_words.astype(np.int32)  # convert visual_words to int32
    for word in visual_words:
        histogram[word] += 1
    return histogram

# Step 5: Classifier Training and Testing
def train_classifier(train_histograms, train_labels):
    svm = LinearSVC()
    svm.fit(train_histograms.astype(np.float32), train_labels.astype(np.float32))
    return svm

def test_classifier(svm, test_histograms, test_labels):
    pred_labels = svm.predict(test_histograms.astype(np.float32))
    print(classification_report(test_labels.astype(np.float32), pred_labels.astype(np.float32)))
    accuracy = accuracy_score(test_labels.astype(np.float32), pred_labels.astype(np.float32))
    return accuracy

# Load train and test images
train_images = X_train
train_labels = y_train

test_images = X_test
test_labels = y_test

# Learn visual words
num_words = 1200
kmeans = learn_visual_words(train_images, num_words)


In [7]:
# Compute histograms for train and test images
train_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in train_images])
test_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in test_images])

# Train and test SVM classifier
svm = train_classifier(train_histograms, train_labels)
accuracy = test_classifier(svm, test_histograms, test_labels)

print("Accuracy: %.2f%%" % (accuracy * 100))

              precision    recall  f1-score   support

         0.0       0.97      0.97      0.97       883
         1.0       0.96      0.95      0.96       966
         2.0       0.98      0.97      0.97       972
         3.0       0.98      0.97      0.98       892
         4.0       0.96      0.96      0.96       901
         5.0       0.97      0.99      0.98       847
         6.0       0.97      0.98      0.97       907
         7.0       0.97      0.98      0.97       851
         8.0       0.97      0.96      0.97       898
         9.0       0.98      0.96      0.97       881
        10.0       0.97      0.97      0.97       934
        11.0       0.98      0.99      0.99       904
        12.0       0.91      0.94      0.92       853
        13.0       0.92      0.91      0.92       900
        14.0       0.94      0.95      0.95       878
        15.0       0.95      0.96      0.96       866
        16.0       0.96      0.97      0.96       873
        17.0       0.94    

In [6]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# Step 1: Feature Extraction
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    # Check if descriptors is not None and has at least one row
    if descriptors is not None and descriptors.shape[0] > 0:
        return descriptors
    else:
        return np.zeros((1, 128), dtype=np.float32) # Return empty array of shape (0, 128)

# Step 2: Feature Clustering
def learn_visual_words(train_images, num_words):
    # Concatenate features from all images
    descriptors = np.concatenate([extract_features(img) for img in train_images])
    # Cluster features into visual words
    kmeans = KMeans(n_clusters=num_words, random_state=0).fit(descriptors.astype(np.float32))
    return kmeans

# Step 3: Feature Quantization
def quantize_features(features, kmeans):
    visual_words = kmeans.predict(features)
    return visual_words.astype(np.int32)

# Step 4: Histogram Creation
def create_histogram(visual_words, num_words):
    histogram = np.zeros(num_words, dtype=np.float32)
    visual_words = visual_words.astype(np.int32)  # convert visual_words to int32
    for word in visual_words:
        histogram[word] += 1
    return histogram

# Step 5: Classifier Training and Testing
def train_classifier(train_histograms, train_labels):
    svm = LinearSVC()
    svm.fit(train_histograms.astype(np.float32), train_labels.astype(np.float32))
    return svm

def test_classifier(svm, test_histograms, test_labels):
    pred_labels = svm.predict(test_histograms.astype(np.float32))
    print(classification_report(test_labels.astype(np.float32), pred_labels.astype(np.float32)))
    accuracy = accuracy_score(test_labels.astype(np.float32), pred_labels.astype(np.float32))
    return accuracy

# Load train and test images
train_images = X_train
train_labels = y_train

test_images = X_test
test_labels = y_test

# Learn visual words
num_words = 1400
kmeans = learn_visual_words(train_images, num_words)


In [7]:
# Compute histograms for train and test images
train_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in train_images])
test_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in test_images])

# Train and test SVM classifier
svm = train_classifier(train_histograms, train_labels)
accuracy = test_classifier(svm, test_histograms, test_labels)

print("Accuracy: %.2f%%" % (accuracy * 100))

              precision    recall  f1-score   support

         0.0       0.96      0.96      0.96       886
         1.0       0.95      0.95      0.95       881
         2.0       0.98      0.98      0.98       897
         3.0       0.98      0.98      0.98       898
         4.0       0.97      0.95      0.96       951
         5.0       0.99      0.99      0.99       926
         6.0       0.98      0.98      0.98       895
         7.0       0.97      0.98      0.97       851
         8.0       0.96      0.97      0.97       883
         9.0       0.98      0.98      0.98       904
        10.0       0.96      0.97      0.97       907
        11.0       0.99      0.99      0.99       847
        12.0       0.94      0.94      0.94       873
        13.0       0.93      0.93      0.93       892
        14.0       0.96      0.96      0.96       878
        15.0       0.97      0.97      0.97       925
        16.0       0.97      0.97      0.97       866
        17.0       0.94    

In [8]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# Step 1: Feature Extraction
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    # Check if descriptors is not None and has at least one row
    if descriptors is not None and descriptors.shape[0] > 0:
        return descriptors
    else:
        return np.zeros((1, 128), dtype=np.float32) # Return empty array of shape (0, 128)

# Step 2: Feature Clustering
def learn_visual_words(train_images, num_words):
    # Concatenate features from all images
    descriptors = np.concatenate([extract_features(img) for img in train_images])
    # Cluster features into visual words
    kmeans = KMeans(n_clusters=num_words, random_state=0).fit(descriptors.astype(np.float32))
    return kmeans

# Step 3: Feature Quantization
def quantize_features(features, kmeans):
    visual_words = kmeans.predict(features)
    return visual_words.astype(np.int32)

# Step 4: Histogram Creation
def create_histogram(visual_words, num_words):
    histogram = np.zeros(num_words, dtype=np.float32)
    visual_words = visual_words.astype(np.int32)  # convert visual_words to int32
    for word in visual_words:
        histogram[word] += 1
    return histogram

# Step 5: Classifier Training and Testing
def train_classifier(train_histograms, train_labels):
    svm = LinearSVC()
    svm.fit(train_histograms.astype(np.float32), train_labels.astype(np.float32))
    return svm

def test_classifier(svm, test_histograms, test_labels):
    pred_labels = svm.predict(test_histograms.astype(np.float32))
    print(classification_report(test_labels.astype(np.float32), pred_labels.astype(np.float32)))
    accuracy = accuracy_score(test_labels.astype(np.float32), pred_labels.astype(np.float32))
    return accuracy

# Load train and test images
train_images = X_train
train_labels = y_train

test_images = X_test
test_labels = y_test

# Learn visual words
num_words = 1600
kmeans = learn_visual_words(train_images, num_words)


In [9]:
# Compute histograms for train and test images
train_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in train_images])
test_histograms = np.array([create_histogram(quantize_features(extract_features(img), kmeans), num_words) for img in test_images])

# Train and test SVM classifier
svm = train_classifier(train_histograms, train_labels)
accuracy = test_classifier(svm, test_histograms, test_labels)

print("Accuracy: %.2f%%" % (accuracy * 100))

              precision    recall  f1-score   support

         0.0       0.97      0.96      0.97       886
         1.0       0.95      0.96      0.96       881
         2.0       0.98      0.98      0.98       897
         3.0       0.99      0.99      0.99       898
         4.0       0.97      0.96      0.97       951
         5.0       0.99      0.99      0.99       926
         6.0       0.98      0.99      0.98       895
         7.0       0.98      0.98      0.98       851
         8.0       0.98      0.98      0.98       883
         9.0       0.99      0.99      0.99       904
        10.0       0.97      0.98      0.98       907
        11.0       0.98      0.99      0.99       847
        12.0       0.94      0.93      0.93       873
        13.0       0.94      0.93      0.94       892
        14.0       0.97      0.97      0.97       878
        15.0       0.98      0.97      0.98       925
        16.0       0.97      0.98      0.97       866
        17.0       0.95    